# Problem Set 5: Neural Nets for Factor Investing

California Institute of Technology

BEM 114: Hedge Funds

Spring 2023


Due: May 23rd, 2023

Please submit your assignments on Canvas before 11:59 p.m. on the due date. Please upload both the Jupyter (iPython) notebook and a PDF of the notebook.  You do not need to submit the data. 



## Introduction
In this problem set you will fit a simple feed-forward Neural Net in order to demonstrate how a neural net can be used to identify dependencies across factors in order to create a higher alpha. 

In question 1 we'll explore the comparison of a neural net to a linear model fit, and demonstrate some of the reasoning why a ReLU activation function is a common choice for neural nets in finance. In question 2 we'll analyze whether lag parameters may help further increase the performance of our model. In question 3 we'll investigate the importance of checking and robustness and reproducibility of the results of our fit.

I have included some sample output as a sanity check for some questions. As discussed in question 3, the optimization of the neural net uses a stochastic component, and thus, your estimates will be slightly different from when you see here.

## Starting Code
You'll be asked to update the following starting code in order to update the model being fit. This will include model parameters, as well as updating the input space.

The `fitting_returns_data` function is the main function that we'll run in order to pull results.

The functions with an `io_` prefix define input and output variables. The functions with a `'model_` prefix define the model fit that we'll be using. 

This problem set will ask you to create your own `io_` and `model_` functions in order to test different modeling possibilities, and to determine the importance of these decisions in the model fitting process.

In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import warnings




# We'll suppress some warnings from tensorflow, but feel free to 
# comment out this line and see some of the efficiency gains we can get!
warnings.filterwarnings("ignore")


# Note: All io_fns and model_fns start with an 'io_' and 'model_' prefix
# respectively



def fitting_returns_data(data_path, 
                         io_fn, 
                         model_fn, 
                         seed = None,
                         print_summary = True):
    """ 
    Fits data to supplied model and provides returns and alpha estimates.
    
        Parameters:
            data_path (str): path to factor data with first column as date
                6 factors, and the last column as risk free rate
            io_fn (fn): a function to calculate input/output pairs for the  
                model fit
            model_fn (fn): a function that takes in input data, output data, 
                and training and testing values, and returns the strategy
                for each day, and the observed return for each day
            seed (int): if given, the seed for the model fit will be set to
                this value
            print_summary (bool): if True, then the model won't print a summary
                of the OLS fit for our strategy return.
        Returns:
            strat_df (pd.DataFrame): The weights we would give each factor
                on each day
            return_vector (list): The returns from our strategy on each day
            model_OLS (RegressionResultsWrapper): A regression calculating the
                alpha for our strategy
    """
    # if given, set the random seed
    if seed:
        set_seed(seed)
        
    # Load the data into a pandas DataFrame
    data = pd.read_csv(data_path)
    # Drop date and risk free rate
    data = data.iloc[:, 1:7]
    
    # Shift the data by one time step to create input/output pairs
    X, y = io_fn(data) 

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    
    # Fit the model
    strat_df, return_vector = model_fn(X, y, X_train, y_train, X_val, y_val)
    
    # Calculate alpha
    y_ols = sm.add_constant(y)
    model_OLS = sm.OLS(return_vector, y_ols).fit()
    if print_summary:
        print(model_OLS.summary())
    
    return strat_df, return_vector, model_OLS


def set_seed(seed):
    # Adding a fixed seed from this solution: https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras
    # 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed)

    # 2. Set the `python` built-in pseudo-random generator at a fixed value
    random.seed(seed)

    # 3. Set the `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed)

    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    tf.compat.v1.set_random_seed(seed)
    return

def io_day_1_lag(data):
    # Create input output pairs, where the input is the previous day of data
    # and the output is the current day of data.
    X = data.shift(1).dropna().reset_index(drop=True)
    y = data.dropna().iloc[1:,:].reset_index(drop=True)
    return X, y

def io_day_1_lag_second_order_input(data):
    # Create input output pairs where input data includes second order interactions
    X, y = io_day_1_lag(data)
    cols = X.columns
    
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            col_name = cols[i] + cols[j]
            col_values = X[cols[i]] * X[cols[j]]
            X[col_name] = col_values
    return X, y

def model_feed_forward(X, y, X_train, y_train, X_val, y_val):
    # Define the neural network model
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(6, activation='linear'))
    
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Train the model, verbose = 0 means reports aren't printed 
    # at the end of each epoch
    model.fit(X_train, y_train, batch_size=32, epochs=50, 
              validation_data=(X_val, y_val), verbose = 0)

    # Make predictions
    predictions = model.predict(X)

    pred_df = pd.DataFrame(predictions)
    
    return predictions_to_returns(pred_df, y)

def predictions_to_returns(pred_df, y):
    # Given the predictions of each factor for each day, calculate our
    # strategy for each day, and the returns for each day
    
    # Apply our strategy to our predictions
    strat_df = pred_df.apply(lambda row : max_predicted_factor_strat(row), axis = 1)

    # Calculate our returns
    return_vector = np.multiply(strat_df,y).apply(sum, axis = 1)
    
    return strat_df, return_vector
    

def max_predicted_factor_strat(row):
    # For each day, set our strategy to be the factor with
    # the highest predicted return
    max_pred_return = max(row)
    row_list = [x == max_pred_return for x in row]
    return pd.Series(row_list)

2023-05-22 21:21:37.344832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Question 0 - Run the Neural Net and Interpet the alpha
The following code fits a feed-forward neural net on all data, and prints a summary.
Run the code and provide an interpretation of the alpha.

```
fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_1_lag, 
    model_feed_forward);
```

**Note**: The test-train split we're employing in these model fits is faulty. We use data from the future to predict our model, so all results here have some level of overfitting. However, I separately fit the models by only using past data, and we were able to get similar results, so the results still seem robust to the potential overfitting.

### Question 0 Solution

In [2]:
# Calculate returns and alphas using feed forward neural net model
fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_1_lag, 
    model_feed_forward);

2023-05-22 21:21:39.290099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


469/469 [==============================] - 0s 307us/step
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     743.9
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        21:21:50   Log-Likelihood:                -16248.
No. Observations:               14998   AIC:                         3.251e+04
Df Residuals:                   14991   BIC:                         3.256e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

The alpha is 0.1170 with a p-value of 0 (to 3 significant figures). The p-value indicates the results are strongly statistically significant at a 1% significance level (and even lower). The alpha itself is positive, indicating that the strategy has a higher average return than expected (i.e., than the market). This alpha is not large, but it is performing well. 

## Question 1 - Comparing Neural Net to OLS

### 1ai [20 points] Set a linear activation function
For the feed-forward neural net that is fit above, the `model_feed_forward` function sets up activation across hidden layers with the following code:

```
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(6,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(6, activation='linear'))
```

Write a new function called `model_feed_forward_linear`, which changes the `'relu'` parameter to `'linear'`. 


Calculate reults for the neural net with linear activation. How does this compare to the ReLU activation from Question 0? 

### Solution 1ai 

In [3]:
# Define linear feed-forward neural net based on model_feed_forward
def model_feed_forward_linear(X, y, X_train, y_train, X_val, y_val):
    # Define the neural network model
    model = Sequential()
    model.add(Dense(32, activation='linear', input_shape=(X.shape[1],)))
    model.add(Dense(16, activation='linear'))
    model.add(Dense(6, activation='linear'))
    
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Train the model, verbose = 0 means reports aren't printed 
    # at the end of each epoch
    model.fit(X_train, y_train, batch_size=32, epochs=50, 
              validation_data=(X_val, y_val), verbose = 0)

    # Make predictions
    predictions = model.predict(X)

    pred_df = pd.DataFrame(predictions)
    
    return predictions_to_returns(pred_df, y)

# Calculate returns and alphas from linear neural net
fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_1_lag, 
    model_feed_forward_linear);

469/469 [==============================] - 0s 334us/step
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     371.3
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        21:22:01   Log-Likelihood:                -15563.
No. Observations:               14998   AIC:                         3.114e+04
Df Residuals:                   14991   BIC:                         3.119e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

This seems to result in lower alpha compared to the ReLU activation from question 0, and is still statistically significant.


### 1aii [10 points] Compare with linear model

The following code defines a function called `model_linear_fit`, which fits a linear model on y_train and X_train and outputs the predicted return given `X`, called `pred_df`. The function then calculates the  returns of the linear model by running `predictions_to_returns(pred_df, y)`. 

Since the neural net is only fitting on linear relationships, we should see similar results across the linear neural net and the linear OLS model here. 

Calculate reults for the linear model below. How does the alpha compare to the linear neural net?  

### Solution 1aii 

In [4]:
# Define linear OLS model
## Linear Model Fit
def model_linear_fit(X, y, X_train, y_train, X_val, y_val):
    model_OLS = sm.OLS(y_train, X_train).fit()
    # Make predictions
    predictions = model_OLS.predict(X)

    pred_df = pd.DataFrame(predictions)
    
    return predictions_to_returns(pred_df, y)

# Calculate returns and alphas using linear OLS model
fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_1_lag, 
    model_linear_fit);

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     336.3
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        21:22:03   Log-Likelihood:                -15641.
No. Observations:               14998   AIC:                         3.130e+04
Df Residuals:                   14991   BIC:                         3.135e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0871      0.006     15.470      0.0

This results in very slighlty higher alpha than the linear neural net, and is also statistically significant.

### 1b [20 points] Include Interaction Terms
Linear models don't account for any interaction effects. In order to account for an interaction we can add input variables that give the product of factor returns on each day. This would be analogous to adding a interaction term to a linear model.

Using the io_ function `io_day_1_lag_second_order_input`, and your `model_linear_fit` function, calculate returns while including second order inputs.

Write a new io_ function `io_day_lag_third_order_input`, to also include third order fits in your input data.

How do these models compare to the ReLU alpha?


### Solution 1b

In [5]:
# Calculate returns and alphas using second order interactions


fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_1_lag_second_order_input, 
    model_linear_fit);

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     523.4
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        21:22:04   Log-Likelihood:                -16011.
No. Observations:               14998   AIC:                         3.204e+04
Df Residuals:                   14991   BIC:                         3.209e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0942      0.006     16.334      0.0

This model results in less alpha than ReLU.

In [6]:
# Define function for calculating third order interactions based on io_day_1_lag_second_order_input
def io_day_lag_third_order_input(data):
    X, y = io_day_1_lag(data)
    cols = X.columns
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            for k in range(j+1, len(cols)):
                col_name = cols[i] + '_' + cols[j] + '_' + cols[k]
                col_values = X[cols[i]] * X[cols[j]] * X[cols[k]]
                X[col_name] = col_values
    return X, y

# Calculate returns and alphas using third order interactions
fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_lag_third_order_input, 
    model_linear_fit);

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     356.7
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        21:22:05   Log-Likelihood:                -15751.
No. Observations:               14998   AIC:                         3.152e+04
Df Residuals:                   14991   BIC:                         3.157e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0995      0.006     17.556      0.0

This model results in less alpha than ReLU.

## Question 2 - Adding Time Lag Parameters
### 2 [25 points] Lag Time Parameters
Our current neural net only uses the past 1 day of data. For time series data, including more lag days can be useful. For example, if a factor return is high for 2 consecutive days, that may be more informative than just knowing that the return was only high for the previous day.

In this question, we will simply add a new column to our input data for each lagged data. This is analogous to an AutoRegressive Model, which is a popular financial engineering tool, [see this textbook](https://link.springer.com/book/10.1007/978-1-4939-2614-5). In the Neural Net literature, a Recurrent Neural Net is a common tool for more directly accounting for time lagged data directly in the neural net architecture, but the lagged model gets us a good amount of the way there!

Starting from the `io_day_1_lag` function write an `io_day_5_lag` function, which adds to the input dataframe 5 days of lagged data per factor. This will mean your input data will now have 6*5 = 30 columns instead of 6 columns.

When complete, run the following and compare results to the original ReLU model from Question 0. How do the results compare? Should we continue to to pursue incorporating lag effects in our analysis of this data?

```
fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_5_lag, 
    model_feed_forward);
 ```
 
 ### Question 2 Solution

In [7]:
# Define function for calculating five-day lags based on io_day_1_lag
def io_day_5_lag(data):
    y = data.iloc[5:,:].reset_index(drop=True)
    X = pd.concat([data.shift(i).rename(columns=lambda x: x+f'_lag{i}') for i in range(1, 6)], axis=1).dropna().reset_index(drop=True)
    return X, y
# Calculate returns and alphas using the feed forward neural net with
# five day lagged input variables.

fitting_returns_data(
    'ff6_factors_19630701_20230131.csv', 
    io_day_5_lag, 
    model_linear_fit);


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     278.1
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        21:22:06   Log-Likelihood:                -15238.
No. Observations:               14994   AIC:                         3.049e+04
Df Residuals:                   14987   BIC:                         3.054e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1056      0.005     19.265      0.0

## Question 3 - Investigating Potentials for P-Hacking
### 3 [25 points] Randomness in Alphas

Neural  nets are fit via a Stochastic Gradient Descent. This implies that there is inherent randomness in any fit of the model. One good way to account for this noise in your model is to refit the model multiple times and observe the distribution. It's more accurate to report the median or mean of these estimates, although it can be hard to tell if a paper/report has cherry-picked the best result in this way.

Using the `io_day_1_lag`, and `model_feed_forward` settings, rerun the model 100 times and get a distribution for the alpha given. (You can use the `seed` parameter in the `fitting_returns_data` function if you want to be able to reproduce a given high return.)

Set `print_summary = False` in `fitting_returns_data` in order to avoid large amounts of output.

What's the highest return you could get if you were to ignore the importance of the robustness of a model result? What would be a downside of reporting a result like this?

Note: running the model 100 times may take awhile (over an hour on Google Colab). Debug your code before attempting the 100 cycles.

### Question 3 Solution


In [8]:
# Rerun model 100 times
alpha_values = []
for i in range(100):
    _, _, model_OLS = fitting_returns_data(
        'ff6_factors_19630701_20230131.csv', 
        io_day_1_lag, 
        model_feed_forward, 
        seed=i, 
        print_summary=False
    )
    alpha_values.append(model_OLS.params[0])  # Assuming alpha is the intercept term


469/469 [==============================] - 0s 316us/step


In [9]:
max(alpha_values)

0.13298627654264922

The highest return you could get is an alpha of 0.1329. The downside of reporting a result like this is that it's likely the result of noise / overfitting, and not reproducable